# Exercice 3 - Merry Christmhash
⚠️ Run this with an ipython kernel, not sage

In [1]:
import binascii
import struct
import operator
import random
import math

## Parameters

In [2]:
# avoids undesired parameters printing when commented
dummy_variable=1

### Tom

In [3]:

Q3a_M="If you are crying, fight against it! If you're regretting, walk forward! Only complaining on your misfortune, you're nothing but a common pig! (Ciel Phantomhive, Kuroshitsuji)"
Q3a_H=[1712935222, 2277794147, 2376157840, 3916784640, 1346939745, 2708679518, 85473800, 3788905359]
Q3a_K=[1943621227, 886196854, 2945353417, 208223340, 1212883955, 2473679821, 3054683082, 2029524489, 3963858188, 3339906788, 2823505815, 3350944332, 312194943, 3986347121, 2144670729, 741633462, 3663151569, 3544144315, 2690702324, 1848842070, 811599001, 510224092, 4281468752, 752009382, 1104123092, 3497858695, 2695667604, 1727413627, 3050579566, 510494887, 4285911224, 952491461, 4237080290, 10651148, 4015126669, 154644693, 1282241408, 2068274028, 2010214286, 3416638395, 2453720085, 3229980118, 2094058516, 1197555596, 922344893, 2154715712, 3908004108, 481673144, 1957904962, 3050061601, 541865778, 1006059256, 585392047, 1178853390, 2805354064, 972068900, 3754759061, 1435964078, 398864381, 2460187893, 1385904313, 3634606251, 2462881886, 3619520975]
Q3a_S=[[22, 16, 27], [21, 4, 23]]
Q3b_t="011e9faabb4e5381"
Q3b_d="384bba4b1dbd84001ccb55791cfcb7ac4da9f6f67174102751830195e3ca5fa6"
Q3b_H=[1410008662, 3165996059, 4265820587, 3228924364, 2876772144, 502571215, 1296440626, 3528054622]
Q3b_K2=[1660732939, 2978614430, 479083038, 4039110515, 503760146, 2160178725, 3371169074, 3023444970, 1832019271, 1110792947, 4275626369, 1675250051, 2784406696, 4257265674, 517985909, 3086983861, 3409296999, 1504386796, 374146206, 4055850561, 2556892257, 2255194630, 2066495719, 259846162, 593904479, 2868322264, 3146981800, 3913516318, 625630895, 1491933359, 1856807117, 3795630429, 3035508049, 3725788811, 2587675021, 4081590610, 1694559383, 2421525974, 2954023209, 3274021365, 556994319, 3840950079, 2748231069, 4293925912, 1934082025, 683379336, 3605902893, 3811110963, 3312404527, 562459431, 3079990482, 4161695928, 1789332738, 3133889965, 94188891, 1773848693]
Q3b_S=[[8, 1, 2], [9, 5, 13]]

dummy_variable=1

### Manon

## $H(m,H,K,S)$ implementation

In [4]:
# https://200ok.ch/posts/2018-12-09_unhexlify.html
def xxd(s):
    return binascii.hexlify(s).decode()

In [395]:
def unxxd(s):
    return binascii.unhexlify(s)

In [115]:
def pack(k,n):
    if k==32:
        return struct.pack('>L',n)
    elif k==64:
        return struct.pack('>Q',n)
    else:
        print("Error, k should be 32 or 64, here it's {}".format(k))
        return None

In [6]:
assert list(pack(32,1))==[0,0,0,1]
assert list(pack(64,1))==[0,0,0,0,0,0,0,1]

In [7]:
# we must make sure ^ and ~ operators are well defined as used later
for i in range(4096):
    r1 = random.randint(0,2**64)
    r2 = random.randint(0,2**64)
    assert r1^r2 == operator.xor(r1,r2)
    assert ~r1 == operator.invert(r1)

In [188]:
 def compress2(x, W, K, S):
    mod = 2^32
    y = x.copy()
    for i in range(64):
        s0 = ((y[0] >> S[0][0]) % mod) | ((y[0] << (32 - S[0][0])) % mod)
        s0 = operator.xor(s0, ((y[0] >> S[0][1]) % mod) | ((y[0] << (32 - S[0][1])) % mod))
        s0 = operator.xor(s0, ((y[0] >> S[0][2]) % mod) | ((y[0] << (32 - S[0][2])) % mod))
        
        s1 = ((y[4] >> S[1][0]) % mod) | ((y[4] << (32 - S[1][0])) % mod)
        s1 = operator.xor(s1, ((y[4] >> S[1][1]) % mod) | ((y[4] << (32 - S[1][1])) % mod))
        s1 = operator.xor(s1, ((y[4] >> S[1][2]) % mod) | ((y[4] << (32 - S[1][2])) % mod))
        
        t1 = operator.xor(y[4] & y[5], ((~y[4])& y[6]))
        t2 = (s1 + t1 + y[7] + K[i] + W[i]) % mod
        t3 = operator.xor(operator.xor(y[0] & y[1], y[0] & y[2]), y[1] & y[2])
        y = [(s0 + t2 + t3) % mod , y[0], y[1], y[2], (y[3] + t2) % mod, y[4], y[5], y[6]]
    return y 

In [198]:
def compression_step(state, k_i, w_i):
    a, b, c, d, e, f, g, h = state
    s1 = rotate_right(e, 6) ^ rotate_right(e, 11) ^ rotate_right(e, 25)
    ch = (e & f) ^ (~e & g)
    tmp1 = (h + s1 + ch + k_i + w_i) & 0xffffffff
    s0 = rotate_right(a, 2) ^ rotate_right(a, 13) ^ rotate_right(a, 22)
    maj = (a & b) ^ (a & c) ^ (b & c)
    tmp2 = (tmp1 + s0 + maj) & 0xffffffff
    tmp3 = (d + tmp1) & 0xffffffff
    return (tmp2, a, b, c, tmp3, e, f, g)

def compression(state, w, round_keys = None):
    if round_keys is None:
      round_keys = self.k
    for i in range(64):
      state = compression_step(state, round_keys[i], w[i])
    return state

In [191]:
def extend2(M):
    print("M: " + str(list(M)))
    W = [0 for i in range(64)]
    for i in range(0,16):
        W[i] = M[i]
    for i in range(16,64):
        a, b = W[i-15], W[i-2]
        s0 = rotate_right(a, 7) ^ rotate_right(a, 18) ^ (a >> 3)
        s1 = rotate_right(b, 17) ^ rotate_right(b, 19) ^ (b >> 10)
        #t1 = (((a>>7) | (a<<25)) ^ (((W[i-15])>>18) | ((W[i-15])<<14)) ^ (((W[i-15])>>3)))%2**32
        #t2 = ((((W[i-2])>>17) | ((W[i-2])<<15)) ^ (((W[i-2])>>19) | ((W[i-2])<<13)) ^ (((W[i-2])>>10)))%2**32
        W[i] = (W[i-16]+W[i-7]+s0+s1) & 0xffffffff
    print("W: " + str(W))
    return W

In [213]:
def oldH(m,H,K,S):
    m = list(m)
    m_numberofbytes = len(m)
    l = (-m_numberofbytes-9)%64
    m = m + [0x80] + [0]*l + list(pack(64,8*m_numberofbytes))
    print("m: " + str(bytes(m)))
    M = list(chunks(m,64))
    d = H
    L = len(M)
    for i in range(L):
        W = extend(M[i])
        y = compress(d,W,K,S)
        #print("d: " + str(d))
        #d = (d+y)
        d = [(di + yi) % 2**32 for di,yi in zip(d,y)]
        #print("y: " + str(y))
        
    d_reduced = d[:8]
    d_out = []
    for di in d_reduced:
         d_out += pack(32,di)
    return d_out

In [318]:
def extend(M):
    mis = list(chunks(bytes(M),4))
    W = [0 for i in range(64)]
    for i in range(0,16):
        W[i] = mis[i][0]<<24|mis[i][1]<<16|mis[i][2]<<8|mis[i][3]
    for i in range(16,64):
        t1 = ((((W[i-15])>>7) | ((W[i-15])<<25)) ^ (((W[i-15])>>18) | ((W[i-15])<<14)) ^ (((W[i-15])>>3)))%2**32
        t2 = ((((W[i-2])>>17) | ((W[i-2])<<15)) ^ (((W[i-2])>>19) | ((W[i-2])<<13)) ^ (((W[i-2])>>10)))%2**32
        W[i] = (W[i-16]+W[i-7]+t1+t2)%2**32
    return W

In [246]:
def compress(x,W,K,S):
    y = x
    for i in range(64):
        s0=0
        for j in range(0,3):
            # we make all our ops modulo 2^32 as y elements are expected to be
            s0 = (s0 ^ ((y[0]>>(S[0][j])) | (y[0]<<(32-S[0][j]))))%2**32
        s1=0
        for j in range(0,3):
            s1 = (s1 ^ ((y[4]>>(S[1][j])) | (y[4]<<(32-S[1][j]))))%2**32
        t1 = (y[4] & y[5] ^ (~y[4]) & y[6])%2**32
        t2 = (s1 + t1 + y[7] + K[i] + W[i])%2**32
        t3 = (y[0] & y[1] ^ y[0] & y[2] ^ y[1] & y[2])%2**32
        
        tmp0 = (s0+t2+t3)%2**32
        tmp4 = (y[3]+t2)%2**32
        y = [tmp0,y[0],y[1],y[2],tmp4,y[4],y[5],y[6]]
    return y

In [190]:
def rotate_right(v, n):
    w = (v >> n) | (v << (32 - n))
    return w & 0xffffffff

In [192]:
def chunks(lis, n):
    """Yield successive n-sized chunks from list."""
    for i in range(0, len(lis), n):
        yield lis[i:i + n]

In [241]:
def H(m,H,K,S):
    m_numberofbytes = len(m)
    l = (-m_numberofbytes-9)%64
    m = bytes(m) + b'\x80'  + b'\x00' *l + pack(64,8*m_numberofbytes)
    M = list(chunks(m,64))
    d = H
    L = len(M)
    for i in range(L):
        W = extend(M[i])
        y = compress(d,W,K,S)
        #print("d: " + str(d))
        #d = (d+y)
        d = [(di + yi) % 2**32 for di,yi in zip(d,y)]
        #print("y: " + str(y))
        
    d_reduced = d[:8]
    d_out = []
    for di in d_reduced:
         d_out += pack(32,di)
    return d_out

In [285]:
digest = H(Q3a_M.encode(), Q3a_H, Q3a_K, Q3a_S)
print(digest,)

M: b"If you are crying, fight against it! If you're regretting, walk "
W: [1231429753, 1869946977, 1919230051, 1920559470, 1730945126, 1768384628, 543254369, 1768846196, 543781921, 541681184, 2037347623, 1919230066, 1701278309, 1953786222, 1730945143, 1634495264, 2231644325, 578924510, 203536228, 2184156885, 148984660, 2847484380, 2125633676, 3149400112, 2146760258, 3459428418, 2219478028, 2757795882, 4010104114, 563488275, 2031116743, 279503366, 1675540125, 3092878061, 2633830211, 1779651646, 922319119, 2488545741, 2534645383, 1965233490, 626001823, 4258324992, 1533116074, 1984645744, 3219482221, 515333783, 1153290782, 364043848, 1573801416, 538587960, 2804713259, 3504135578, 2933624647, 3368663638, 3527527148, 4165753813, 1212883169, 1276917076, 1568071280, 1448250415, 937352231, 1226461067, 397669295, 3566521527]
M: b"forward! Only complaining on your misfortune, you're nothing but"
W: [1718579831, 1634886689, 542076524, 2032165743, 1836084321, 1768843630, 1730178926, 544829301, 191

In [239]:
Q3a_d = binascii.hexlify(bytes(digest)).decode()
Q3a_d

'9daa416bfe8241565468cf90dc043b37990ee22acd68569851aa7e9b7dcea627'

In [293]:
digest

[157,
 170,
 65,
 107,
 254,
 130,
 65,
 86,
 84,
 104,
 207,
 144,
 220,
 4,
 59,
 55,
 153,
 14,
 226,
 42,
 205,
 104,
 86,
 152,
 81,
 170,
 126,
 155,
 125,
 206,
 166,
 39]

In [243]:
test_msg = 'Encoded with random keys'
tdigest = H(test_msg.encode(), Q3a_H, Q3a_K, Q3a_S)
print(tdigest,)
print(binascii.hexlify(bytes(tdigest)).decode())

M: b'Encoded with random keys\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc0'
W: [1164862319, 1684366368, 2003399784, 544366958, 1685024032, 1801812339, 2147483648, 0, 0, 0, 0, 0, 0, 0, 0, 192, 1522197188, 3891742175, 3836386829, 32341671, 928288908, 2364323079, 1515866404, 649785226, 1435989715, 250124094, 1469326411, 2429553944, 598071608, 1634056085, 4271828083, 4262132921, 2272436470, 39791740, 2337714294, 3555435891, 1519859327, 57013755, 2177157937, 1679613557, 2900649386, 612096658, 172526146, 2214036567, 3330460486, 1490972443, 1925782519, 4215628757, 2379791427, 2058888203, 1834962275, 3917548225, 2375084030, 1546202149, 3188006334, 4280719833, 726047027, 3650106516, 4058756591, 1443098026, 1972312730, 1218108430, 3428722156, 366022263]
[151, 26, 88, 10, 144, 240, 17, 167, 107, 126, 133, 20, 207, 23, 153, 249, 53, 200, 15, 35, 133, 51, 192, 102, 171, 246, 100, 178, 

In [121]:
m = Q3a_M.encode()
m = 'Encoded with random keys'.encode()
m_numberofbytes = len(m)
l = (-m_numberofbytes-9)%64
m = bytes(m) + b'\x80'  + b'\x00' *l + pack(64,8*m_numberofbytes)
m

b'Encoded with random keys\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc0'

## Key recovery

### Invert Compression

our notation <=> ctf notation
tmp0 = tmp2
tmp4 = tmp3 
t1 = ch 
t2 = tmp1 
t3 = maj

The compression step goes as follows:

```
y = x
[computation]
return  (tmp_0, y_0, y_1, y_2, tmp_4, y_4, y_5, y_6)
```

So we need to recover $y_3$ and $y_7$.

We know that :
$tmp_0 = (s_0+t_2+t_3)\ mod\ 2^{32}
\\tmp_4 = (y_3+t_2)\ mod\ 2^{32}$

So we can find $y_3$: 
$t_2 = (tmp_0 - (s_0 + t_3))\ mod\ 2^{32}
\\y_3 = (tmp_4-tmp_0)\ mod\ 2^{32}$

And finally $y_7$:
$t_2 = (s_1 + t_1 + y_7 + k_i + w_i)\ mod\ 2^{32}
\\t_2 = (tmp_0 - (s_0 + t_3)) \ mod\ 2^{32}
\\y_7 = (t_2 - (k_i + s_1 + t_1 + w_i))\ mod\ 2^{32} $

All of these values are based on parameters we know, expect for recovering $y_7$ which requires us to know $k_i$ (which is fine for the upper rounds but an issue for the first 8 $k_i$ values which we are trying to uncover). 

Let's first invert the upper rounds:

In [372]:
def invert_compression_step(y,wi,ki,S):
    tmp0,y0,y1,y2,tmp4,y4,y5,y6 = y
    
    t3 = (y0 & y1 ^ y0 & y2 ^ y1 & y2)%2**32
    
    s0=0
    for j in range(0,3):
        # we make all our ops modulo 2^32 as y elements are expected to be
        s0 = (s0 ^ ((y0>>(S[0][j])) | (y0<<(32-S[0][j]))))%2**32
    s1=0
    for j in range(0,3):
        s1 = (s1 ^ ((y4>>(S[1][j])) | (y4<<(32-S[1][j]))))%2**32
        
    t2 = (tmp0 - (s0 + t3))%2**32
    
    y3 = (tmp4 - t2) %2**32
    
    t1 = (y4 & y5 ^ (~y4) & y6)%2**32
    
    y7 = (t2 - (ki + s1 + t1 + wi))%2**32
           
    og_y = y0, y1, y2, y3, y4, y5, y6, y7
    
    #print(og_y)
          
    return og_y

### Recover keys 

In [412]:
def recover_keys(digest, t, K2, H, S):
    chunksofdigest = list(chunks(digest,8))
    digest_as_int_array = [int.from_bytes(binascii.unhexlify(d), "big") for d in chunksofdigest]
    print(digest_as_int_array)
    
    raw_state = [(d - y)  %2**32 for d, y in zip(digest_as_int_array, H)]
    
    # Padding
    t_numberofbytes = len(t)
    l = (-t_numberofbytes-9)%64
    t = bytes(t) + b'\x80'  + b'\x00' *l + pack(64,8*t_numberofbytes)
    w = extend(t)
    
    secret_keys = 8
    keys = []
    state = raw_state
    original_keys = K2
    for i in range(1, 65 - secret_keys):
        state = invert_compression_step(state, w[-i], original_keys[-i], S)

    print(state)
    f_state = state
    for i in range(7, -1, -1):
        for j in range(i + 1):
            state = invert_compression_step(state, 0, w[i - j], S)
        real_key = state[7 - i] - H[7 - i]
        keys.append(real_key & 0xffffffff)

        state = f_state
        for idx, k in enumerate(keys):
            state = invert_compression_step(state, k, w[7 - idx], S)
    return keys[::-1]

In [413]:
secret = recover_keys(Q3b_d.encode(),unxxd(Q3b_t),Q3b_K2, Q3b_H,Q3b_S)

[944486987, 498959360, 483087737, 486324140, 1302984438, 1903431719, 1367540117, 3821690790]
(3996966641, 305766026, 1972540380, 2017243241, 3885947749, 2692629861, 984687947, 851447517)


In [414]:
secret

[2593449878,
 957238634,
 3764295242,
 3223161263,
 435540614,
 603796397,
 2724912648,
 884172017]

In [415]:
Q3b_t="011e9faabb4e5381"
Q3b_d="384bba4b1dbd84001ccb55791cfcb7ac4da9f6f67174102751830195e3ca5fa6"
Q3b_H=[1410008662, 3165996059, 4265820587, 3228924364, 2876772144, 502571215, 1296440626, 3528054622]
Q3b_K2=[1660732939, 2978614430, 479083038, 4039110515, 503760146, 2160178725, 3371169074, 3023444970, 1832019271, 1110792947, 4275626369, 1675250051, 2784406696, 4257265674, 517985909, 3086983861, 3409296999, 1504386796, 374146206, 4055850561, 2556892257, 2255194630, 2066495719, 259846162, 593904479, 2868322264, 3146981800, 3913516318, 625630895, 1491933359, 1856807117, 3795630429, 3035508049, 3725788811, 2587675021, 4081590610, 1694559383, 2421525974, 2954023209, 3274021365, 556994319, 3840950079, 2748231069, 4293925912, 1934082025, 683379336, 3605902893, 3811110963, 3312404527, 562459431, 3079990482, 4161695928, 1789332738, 3133889965, 94188891, 1773848693]
Q3b_S=[[8, 1, 2], [9, 5, 13]]

In [416]:
AllKeys = secret + Q3b_K2
len(AllKeys)

64

In [417]:
recoverdigest = H(unxxd(Q3b_t), Q3b_H, AllKeys, Q3b_S)
print(recoverdigest,)

[56, 75, 186, 75, 29, 189, 132, 0, 28, 203, 85, 121, 28, 252, 183, 172, 77, 169, 246, 246, 113, 116, 16, 39, 81, 131, 1, 149, 227, 202, 95, 166]


In [418]:
shouldbed = binascii.hexlify(bytes(recoverdigest)).decode()
shouldbed

'384bba4b1dbd84001ccb55791cfcb7ac4da9f6f67174102751830195e3ca5fa6'

In [388]:
assert(shouldbed == Q3b_d)